# API Scavenger Game

## Challenge 1: Fork Languages

You will find out how many programming languages are used among all the forks created from the main lab repo of your bootcamp.

In [131]:
# import libraries here
import json
import requests
import pandas as pd
import numpy as np
import base64

In [2]:
username = 'SvetlanaGruzdeva'
token = '5fa5888d2bb91f87183edcd0e4efcce08b3d44d3'
base_url = 'https://api.github.com'
owner = 'jmolendijk'
repo = 'dataV2-labs'

Assuming the main lab repo is ironhack-datalabs/madrid-oct-2018, you will:

#### 1. Obtain the full list of forks created from the main lab repo via Github API.

To list forks, we can use the GET method. As explained in the GitHub API documentation, we need to make the request to: GET /repos/:owner/:repo/forks.

In [3]:
# your code here
response = requests.get(f'{base_url}/repos/{owner}/{repo}/forks', auth=(username,token))
data = pd.DataFrame(response.json())
col = list(data.columns.values)
col

['id',
 'node_id',
 'name',
 'full_name',
 'private',
 'owner',
 'html_url',
 'description',
 'fork',
 'url',
 'forks_url',
 'keys_url',
 'collaborators_url',
 'teams_url',
 'hooks_url',
 'issue_events_url',
 'events_url',
 'assignees_url',
 'branches_url',
 'tags_url',
 'blobs_url',
 'git_tags_url',
 'git_refs_url',
 'trees_url',
 'statuses_url',
 'languages_url',
 'stargazers_url',
 'contributors_url',
 'subscribers_url',
 'subscription_url',
 'commits_url',
 'git_commits_url',
 'comments_url',
 'issue_comment_url',
 'contents_url',
 'compare_url',
 'merges_url',
 'archive_url',
 'downloads_url',
 'issues_url',
 'pulls_url',
 'milestones_url',
 'notifications_url',
 'labels_url',
 'releases_url',
 'deployments_url',
 'created_at',
 'updated_at',
 'pushed_at',
 'git_url',
 'ssh_url',
 'clone_url',
 'svn_url',
 'homepage',
 'size',
 'stargazers_count',
 'watchers_count',
 'language',
 'has_issues',
 'has_projects',
 'has_downloads',
 'has_wiki',
 'has_pages',
 'forks_count',
 'mirror_u

#### 2. Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.
Hint: Each language should appear only once in your array.
Print the language array. It should be something like: ["Python", "Jupyter Notebook", "HTML"]

In [4]:
data['language']

0     Jupyter Notebook
1     Jupyter Notebook
2     Jupyter Notebook
3     Jupyter Notebook
4     Jupyter Notebook
5     Jupyter Notebook
6     Jupyter Notebook
7     Jupyter Notebook
8     Jupyter Notebook
9     Jupyter Notebook
10    Jupyter Notebook
11    Jupyter Notebook
12    Jupyter Notebook
Name: language, dtype: object

In [5]:
# your code here
lang = data['language'].unique()
lang

array(['Jupyter Notebook'], dtype=object)

## Challenge 2: Count Commits
Count how many commits were made in the month of october of 2018.
#### 1. Obtain all the commits made in October 2018 via API, which is a JSON array that contains multiple commit objects.

In [6]:
# your code here
response = requests.get(f'{base_url}/repos/{owner}/{repo}/commits?since=2020-07-01T00:00:00Z&until=2020-07-31T00:00:00Z', auth=(username,token))
data = pd.DataFrame(response.json())
col = list(data.columns.values)
data['commit'][0]['author']['date']
data

,sha,node_id,commit,url,html_url,comments_url,author,committer,parents
0,80189cce124398781df93e115bf60755e3ed4d7b,MDY6Q29tbWl0MjgyNjY1MjcyOjgwMTg5Y2NlMTI0Mzk4Nz...,"{'author': {'name': 'Jan Molendijk', 'email': ...",https://api.github.com/repos/jmolendijk/dataV2...,https://github.com/jmolendijk/dataV2-labs/comm...,https://api.github.com/repos/jmolendijk/dataV2...,"{'login': 'jmolendijk', 'id': 54313084, 'node_...","{'login': 'jmolendijk', 'id': 54313084, 'node_...",[{'sha': '820a60e936eb06c25271744ac773d4179950...
1,820a60e936eb06c25271744ac773d41799501388,MDY6Q29tbWl0MjgyNjY1MjcyOjgyMGE2MGU5MzZlYjA2Yz...,"{'author': {'name': 'jmolendijk', 'email': '54...",https://api.github.com/repos/jmolendijk/dataV2...,https://github.com/jmolendijk/dataV2-labs/comm...,https://api.github.com/repos/jmolendijk/dataV2...,"{'login': 'jmolendijk', 'id': 54313084, 'node_...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[]


#### 2. Count how many commit objects are contained in the array.

In [7]:
# your code here
response = requests.get(f'{base_url}/repos/{owner}/{repo}/commits?', auth=(username,token))
data = pd.DataFrame(response.json())
data['sha'].count()
data

,sha,node_id,commit,url,html_url,comments_url,author,committer,parents
0,732d41f99c3460826fc321566ca79cb2f8bfb2b3,MDY6Q29tbWl0MjgyNjY1MjcyOjczMmQ0MWY5OWMzNDYwOD...,"{'author': {'name': 'Jan Molendijk', 'email': ...",https://api.github.com/repos/jmolendijk/dataV2...,https://github.com/jmolendijk/dataV2-labs/comm...,https://api.github.com/repos/jmolendijk/dataV2...,"{'login': 'jmolendijk', 'id': 54313084, 'node_...","{'login': 'jmolendijk', 'id': 54313084, 'node_...",[{'sha': '80189cce124398781df93e115bf60755e3ed...
1,80189cce124398781df93e115bf60755e3ed4d7b,MDY6Q29tbWl0MjgyNjY1MjcyOjgwMTg5Y2NlMTI0Mzk4Nz...,"{'author': {'name': 'Jan Molendijk', 'email': ...",https://api.github.com/repos/jmolendijk/dataV2...,https://github.com/jmolendijk/dataV2-labs/comm...,https://api.github.com/repos/jmolendijk/dataV2...,"{'login': 'jmolendijk', 'id': 54313084, 'node_...","{'login': 'jmolendijk', 'id': 54313084, 'node_...",[{'sha': '820a60e936eb06c25271744ac773d4179950...
2,820a60e936eb06c25271744ac773d41799501388,MDY6Q29tbWl0MjgyNjY1MjcyOjgyMGE2MGU5MzZlYjA2Yz...,"{'author': {'name': 'jmolendijk', 'email': '54...",https://api.github.com/repos/jmolendijk/dataV2...,https://github.com/jmolendijk/dataV2-labs/comm...,https://api.github.com/repos/jmolendijk/dataV2...,"{'login': 'jmolendijk', 'id': 54313084, 'node_...","{'login': 'web-flow', 'id': 19864447, 'node_id...",[]


## Challenge 3: Hidden Cold Joke

Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

#### 1. Find the secret files.

In [8]:
# your code here
response = requests.get(f'{base_url}/repos/ironhack-datalabs/scavenger/contents', auth=(username,token))
response.json()
data = pd.DataFrame(response.json())
path = data['path'].tolist()
path = path[1:]
path

['15024',
 '15534',
 '17020',
 '30351',
 '40303',
 '44639',
 '45525',
 '47222',
 '47830',
 '49418',
 '50896',
 '55417',
 '55685',
 '60224',
 '64880',
 '66032',
 '68848',
 '70751',
 '70985',
 '88596',
 '89046',
 '89338',
 '91701',
 '97881',
 '98750']

In [103]:
a = pd.DataFrame({'no need' : []})
for i in path:
    resp1 = requests.get(f'{base_url}/repos/ironhack-datalabs/scavenger/contents/{i}', auth=(username,token))
    a = a.append(pd.DataFrame(resp1.json())['name'])
a

,no need,0,1,2,3
name,NaN,.0006.scavengerhunt,40,74,99
name,NaN,.0008.scavengerhunt,.0012.scavengerhunt,2,98
name,NaN,.0007.scavengerhunt,46,97,NaN
name,NaN,.0021.scavengerhunt,22,96,NaN
name,NaN,.0022.scavengerhunt,51,65,NaN
name,NaN,.0005.scavengerhunt,76,91,NaN
name,NaN,.0018.scavengerhunt,67,90,NaN
name,NaN,.0016.scavengerhunt,.0024.scavengerhunt,7,89
name,NaN,.0010.scavengerhunt,66,85,NaN
name,NaN,.0014.scavengerhunt,30,38,84


In [108]:
# Almost in the end of the task I realised that it will be handy to have a dictionary with path as a key and required files as
# values. Couldn't finish it though so had to look for work around anyway.
# temp = a.copy()
# temp['path'] = pd.Series(path).values
# temp = temp.drop(columns=['no need', 2, 3])
# temp
# z1 = list(zip(temp[0], temp[1]))
# z2 = dict(zip(temp['path'], z1))
# z2

{'15024': ('.0006.scavengerhunt', '40'),
 '15534': ('.0008.scavengerhunt', '.0012.scavengerhunt'),
 '17020': ('.0007.scavengerhunt', '46'),
 '30351': ('.0021.scavengerhunt', '22'),
 '40303': ('.0022.scavengerhunt', '51'),
 '44639': ('.0005.scavengerhunt', '76'),
 '45525': ('.0018.scavengerhunt', '67'),
 '47222': ('.0016.scavengerhunt', '.0024.scavengerhunt'),
 '47830': ('.0010.scavengerhunt', '66'),
 '49418': ('.0014.scavengerhunt', '30'),
 '50896': ('.0011.scavengerhunt', '4'),
 '55417': ('.0023.scavengerhunt', '14'),
 '55685': ('.0020.scavengerhunt', '5'),
 '60224': ('.0003.scavengerhunt', '1'),
 '64880': ('3', '57'),
 '66032': ('28', '47'),
 '68848': ('.0004.scavengerhunt', '16'),
 '70751': ('.0019.scavengerhunt', '18'),
 '70985': ('.0017.scavengerhunt', '11'),
 '88596': ('.0002.scavengerhunt', '12'),
 '89046': ('64', '70'),
 '89338': ('.0013.scavengerhunt', '35'),
 '91701': ('.0015.scavengerhunt', '33'),
 '97881': ('.0009.scavengerhunt', '26'),
 '98750': ('.0001.scavengerhunt', '20

In [128]:
# z3 = {}
# for key, value in z2.items():
#     for i in range(0,2):
#         if value[i].endswith('.scavengerhunt'):
#             z3[key] = value[i]
# z3

{'15024': '.0006.scavengerhunt',
 '15534': '.0012.scavengerhunt',
 '17020': '.0007.scavengerhunt',
 '30351': '.0021.scavengerhunt',
 '40303': '.0022.scavengerhunt',
 '44639': '.0005.scavengerhunt',
 '45525': '.0018.scavengerhunt',
 '47222': '.0024.scavengerhunt',
 '47830': '.0010.scavengerhunt',
 '49418': '.0014.scavengerhunt',
 '50896': '.0011.scavengerhunt',
 '55417': '.0023.scavengerhunt',
 '55685': '.0020.scavengerhunt',
 '60224': '.0003.scavengerhunt',
 '68848': '.0004.scavengerhunt',
 '70751': '.0019.scavengerhunt',
 '70985': '.0017.scavengerhunt',
 '88596': '.0002.scavengerhunt',
 '89338': '.0013.scavengerhunt',
 '91701': '.0015.scavengerhunt',
 '97881': '.0009.scavengerhunt',
 '98750': '.0001.scavengerhunt'}

#### 2.  Sort the filenames ascendingly.

In [41]:
# your code here
files = sorted([name for name in (a[0].tolist() + a[1].tolist()) if name.endswith('.scavengerhunt')])

files

['.0001.scavengerhunt',
 '.0002.scavengerhunt',
 '.0003.scavengerhunt',
 '.0004.scavengerhunt',
 '.0005.scavengerhunt',
 '.0006.scavengerhunt',
 '.0007.scavengerhunt',
 '.0008.scavengerhunt',
 '.0009.scavengerhunt',
 '.0010.scavengerhunt',
 '.0011.scavengerhunt',
 '.0012.scavengerhunt',
 '.0013.scavengerhunt',
 '.0014.scavengerhunt',
 '.0015.scavengerhunt',
 '.0016.scavengerhunt',
 '.0017.scavengerhunt',
 '.0018.scavengerhunt',
 '.0019.scavengerhunt',
 '.0020.scavengerhunt',
 '.0021.scavengerhunt',
 '.0022.scavengerhunt',
 '.0023.scavengerhunt',
 '.0024.scavengerhunt']

#### 3. Read the content of each secret files into an array of strings.
Since the response is encoded, you will need to send the following information in the header of your request:
````python
headers = {'Accept': 'application/vnd.github.v3.raw'}
````

In [137]:
# your code here
import base64
resp2 = requests.get(f'{base_url}/repos/ironhack-datalabs/scavenger/contents/44639/.0005.scavengerhunt', auth=(username,token), headers = {'Accept': 'application/vnd.github.v3.raw'})
# resp2.content.decode('ascii')
resp3 = requests.get(f'{base_url}/repos/ironhack-datalabs/scavenger/contents/44639/.0005.scavengerhunt', auth=(username,token))
resp3

<Response [200]>

In [93]:
print(path)
print(files)

['15024', '15534', '17020', '30351', '40303', '44639', '45525', '47222', '47830', '49418', '50896', '55417', '55685', '60224', '64880', '66032', '68848', '70751', '70985', '88596', '89046', '89338', '91701', '97881', '98750']
['.0001.scavengerhunt', '.0002.scavengerhunt', '.0003.scavengerhunt', '.0004.scavengerhunt', '.0005.scavengerhunt', '.0006.scavengerhunt', '.0007.scavengerhunt', '.0008.scavengerhunt', '.0009.scavengerhunt', '.0010.scavengerhunt', '.0011.scavengerhunt', '.0012.scavengerhunt', '.0013.scavengerhunt', '.0014.scavengerhunt', '.0015.scavengerhunt', '.0016.scavengerhunt', '.0017.scavengerhunt', '.0018.scavengerhunt', '.0019.scavengerhunt', '.0020.scavengerhunt', '.0021.scavengerhunt', '.0022.scavengerhunt', '.0023.scavengerhunt', '.0024.scavengerhunt']


In [156]:
import re
q = ['15024', '44639']
w = ['.0005.scavengerhunt']
error = '{"message":"Not Found","documentation_url":"https://docs.github.com/rest/reference/repos#get-repository-content"}'

my_dict = {}
for p in q:
    for f in w:
#         print(p, type(p))
#         print(f, type(f))
        resp2 = requests.get(f'{base_url}/repos/ironhack-datalabs/scavenger/contents/{p}/{f}', auth=(username,token), headers = {'Accept': 'application/vnd.github.v3.raw'})
        if resp2.content.decode('ascii') != error:
            print(resp2.content.decode('ascii'))

percent



In [159]:
# To run this request takes veeeeery long time but I couldn't figure out how to read file without looping through folders.
# And to do this efficient I couldn't figure out how to put folders and files in dictionary.
my_dict = {}
error = '{"message":"Not Found","documentation_url":"https://docs.github.com/rest/reference/repos#get-repository-content"}'
for p in path:
    for f in files:
#         print(p, type(p))
#         print(f, type(f))
        resp2 = requests.get(f'{base_url}/repos/ironhack-datalabs/scavenger/contents/{p}/{f}', auth=(username,token), headers = {'Accept': 'application/vnd.github.v3.raw'})
        if resp2.content.decode('ascii') != error:
            my_dict[f] = resp2.content.decode('ascii')
print(my_dict)

{'.0006.scavengerhunt': 'of\n', '.0008.scavengerhunt': 'spent\n', '.0012.scavengerhunt': '20\n', '.0007.scavengerhunt': 'time\n', '.0021.scavengerhunt': 'need\n', '.0022.scavengerhunt': 'to\n', '.0005.scavengerhunt': 'percent\n', '.0018.scavengerhunt': 'complaining\n', '.0016.scavengerhunt': 'is\n', '.0024.scavengerhunt': 'data.\n', '.0010.scavengerhunt': 'preparing\n', '.0014.scavengerhunt': 'of\n', '.0011.scavengerhunt': 'data,\n', '.0023.scavengerhunt': 'prepare\n', '.0020.scavengerhunt': 'the\n', '.0003.scavengerhunt': 'science,\n', '.0004.scavengerhunt': '80\n', '.0019.scavengerhunt': 'about\n', '.0017.scavengerhunt': 'spent\n', '.0002.scavengerhunt': 'data\n', '.0013.scavengerhunt': 'percent\n', '.0015.scavengerhunt': 'time\n', '.0009.scavengerhunt': 'is\n', '.0001.scavengerhunt': 'In\n'}


In [172]:
sorted_d = sorted(my_dict.items())
my_string = []
for i in sorted_d:
    y = i[1].replace('\n', '')
    my_string.append(y)
my_string

['In',
 'data',
 'science,',
 '80',
 'percent',
 'of',
 'time',
 'spent',
 'is',
 'preparing',
 'data,',
 '20',
 'percent',
 'of',
 'time',
 'is',
 'spent',
 'complaining',
 'about',
 'the',
 'need',
 'to',
 'prepare',
 'data.']

#### 4. Concatenate the strings in the array separating each two with a whitespace.

In [174]:
# your code here
' '.join(my_string)

'In data science, 80 percent of time spent is preparing data, 20 percent of time is spent complaining about the need to prepare data.'

#### 5. Print out the joke.

In [ ]:
# your code here